In [1]:
import torch
import pickle
import random
import torchvision
import numpy as np
import datetime as dt
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms

from avalanche.training.supervised import GSS_greedy
from avalanche.logging import InteractiveLogger, TensorboardLogger
from avalanche.benchmarks.datasets import CIFAR10
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceTime, EpochAccuracy

/home/cal-05/anaconda3/envs/hspark/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
cudnn.enabled = False
cudnn.deterministic = True

In [3]:
train_set = CIFAR10('../Dataset/CIFAR10', train=True, download=True)
test_set = CIFAR10('../Dataset/CIFAR10', train=False, download=True)

transforms_group = dict(
       train=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       ),
       eval=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       )
)

train_set = AvalancheDataset(train_set, transform_groups=transforms_group, initial_transform_group="train")
test_set = AvalancheDataset(test_set, transform_groups=transforms_group, initial_transform_group="eval")

Files already downloaded and verified
Files already downloaded and verified


In [4]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

num_class = 10
incremental = 5

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=False,
                        seed=seed,
                        shuffle=False
                        )

model = torchvision.models.resnet18(pretrained=False, num_classes=num_class)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=1e-2)
criterion = torch.nn.CrossEntropyLoss()

In [5]:
date = dt.datetime.now()
date = date.strftime("%Y_%m_%d_%H_%M_%S")

interactive_logger = InteractiveLogger()
tensor_logger = TensorboardLogger("logs_GSS-Greedy_cifar10_avalanche_" + date)
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceTime(),
    loggers=[interactive_logger, tensor_logger])

2022-08-10 16:55:17.723806: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/cal-05/anaconda3/envs/hspark/lib/python3.8/site-packages/avalanche/training/plugins/evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


## GSS-Greedy

In [6]:
train_batch = 10
eval_batch = 10
epoch = 10

strategies = GSS_greedy(model, optimizer, criterion, mem_size=200, mem_strength=10, input_size=[3, 32, 32], train_epochs=epoch, device=device, train_mb_size=train_batch, eval_mb_size=eval_batch, evaluator=eval_plugin)

In [7]:
print("Starting experiment...")
results = []

for experience in scenario.train_stream:
    print("Start training on experience ", experience.current_experience)
    strategies.train(experience)
    print("End training on experience ", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(strategies.eval(scenario.test_stream[:]))

Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
100%|██████████| 1000/1000 [04:31<00:00,  3.69it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7714
100%|██████████| 1000/1000 [04:24<00:00,  3.78it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8570
100%|██████████| 1000/1000 [04:38<00:00,  3.59it/s]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8999
100%|██████████| 1000/1000 [04:28<00:00,  3.72it/s]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9078
100%|██████████| 1000/1000 [04:41<00:00,  3.55it/s]
Epoch 4 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9198
100%|██████████| 1000/1000 [04:31<00:00,  3.69it/s]
Epoch 5 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9331
100%|██████████| 1000/1000 [04:40<00:00,  3.56it/s]
Epoch 6 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9465
100%|██████████| 1000/1000 [05:11<